# HW 2 - Разложение матриц градиентным методом

Цель задания: В ходе реализации [разложения Таккера](https://proceedings.neurips.cc/paper/2018/file/45a766fa266ea2ebeb6680fa139d2a3d-Paper.pdf) градиентным методом освоить pyTorch и реализовать подходы оптимизации параметров модели (в отсутствии готовых решений).

[Более-менее внятное описание алгоритма канонического разложения](https://www.alexejgossmann.com/tensor_decomposition_tucker/) - само аналитическое разложение вам реализовывать НЕ НУЖНО

In [82]:
import random
import time
import torch
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from numpy.linalg import svd, matrix_rank, pinv, inv
from scipy.linalg import eigh, eig
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from torch import nn

import tensorly as tl
from tensorly.decomposition import tucker

torch.manual_seed(0)

## 1 Создайте 3х мерный тензор
Размер тензора не меньше 100 по каждой из размерностей.

Заполните случайными целыми числами в диапазоне от 0 до 9.

Примечание: разложение будет корректно работать со случайным тензором, только если изначально создавать случайные ядро и матрицы, а потом по ним формировать тензор. Работайте с типом *torch.Tensor.double*.

In [83]:
# Создадим тензор: размер тензора и r задаётся
def get_tensor(size=(100,200,150), r=(10, 10, 10)):
  # data - тензор с заданной размерностью
  # U - список матриц
  # G - ядро разложения
    
  U = [torch.randn(size[i], r[i], dtype=torch.double) for i in range(len(size))]
  
  G = torch.randn(*r, dtype=torch.double)
  
  data = G.clone()
  for i in range(len(size)):
      data = torch.tensordot(data, U[i], dims=([0], [1]))
  
  return data, U, G

Сгенерируйте тензор и добавьте к нему случайный шум с размерностью *1e-2*

In [84]:
size = (100, 200, 300)
r=(10, 10, 10)

data, U, G = get_tensor(size, r)
data.shape, [u.shape for u in U], G.shape

(torch.Size([100, 200, 300]),
 [torch.Size([100, 10]), torch.Size([200, 10]), torch.Size([300, 10])],
 torch.Size([10, 10, 10]))

In [85]:
noise = torch.randn_like(data) * 1e-2
data_noisy = data + noise
data_noisy = (data_noisy - data_noisy.mean()) / data_noisy.std()


Вопрос:
Почему задание не имеет смысла для полностью случайного тензора и зачем добавлять шум? *не отвечать нельзя*

Ответ:

    - Если тензор полностью случайный, то теряется цель разложения — выделение закономерностей и зависимостей.
    - Шум добавляют, чтобы имитировать реальные данные с погрешностями и сделать алгоритм более устойчивым. 

## 2 Реализуйте метод для восстановления тензора по разложению

In [87]:
# Функция, восстанавливающая тензор по ядру и матрицам
def repair_tensor(G_, U):
  # data - восстановленный тензор из матриц и ядра
  # U - список матриц
  # G_ - ядро разложения
  
  data = G_.clone()
  for i in range(len(U)):
      data = torch.tensordot(data, U[i], dims=([0], [1]))
  return data

In [88]:
restored_data = repair_tensor(G, U)

print(restored_data.shape)

mse = torch.mean((restored_data - data) ** 2).item()
print(f"MSE: {mse}")


torch.Size([100, 200, 300])
MSE: 0.0


## 3 Сделайте разложение библиотечным методом
Пакет можете брать любой

In [89]:
tl_data = tl.tensor(data_noisy.numpy())

rank = (10, 10, 10)
core, factors = tucker(tl_data, rank)

restored_tl_data = tl.tucker_to_tensor((core, factors))

restored_torch_data = torch.tensor(restored_tl_data, dtype=torch.double)

Не забудьте померить ошибку разложения по метрике MSE

In [90]:
mse = mean_squared_error(data_noisy.flatten().numpy(), restored_torch_data.flatten().numpy())

print(f"MSE между оригинальным и восстановленным тензором: {mse}")


MSE между оригинальным и восстановленным тензором: 1.0347827097420307e-07


## 4 Реализуйте разложение градиентным методом

### 4.1 Реализуйте *optimizer*
Можно взять из исходников *PyTorch* и отнаследоваться от *torch.optim.optimizer*.
Используйте квадратичный *Loss*.

In [91]:
import math
import torch
from torch.optim.optimizer import Optimizer

class Opt(Optimizer):
    def __init__(self, params, lr=1e-3):

        defaults = dict(lr=lr)
        super().__init__(params, defaults)
        
    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for param in group['params']:
                if param.grad is None:
                    continue

                grad = param.grad.data

                param.data -= group['lr'] * grad

        return loss


### 4.2 Реализуйте цикл оптимизации параметров

Стоит параметры оптимизировать сразу на GPU

In [92]:
def optimize_tucker(data, rank, num_epochs=1000, lr=1e-3, device="cpu"):
    data = data.to(device)
    
    G = torch.randn(*rank, dtype=torch.double, requires_grad=True, device=device)
    U = [torch.randn(data.size(i), rank[i], dtype=torch.double, requires_grad=True, device=device) 
         for i in range(len(rank))]
    
    params = [G] + U

    optimizer = Opt(params, lr=lr)

    loss_fn = torch.nn.MSELoss()

    for epoch in range(num_epochs):
        def closure():
            optimizer.zero_grad()

            reconstructed = G.clone()
            for i in range(len(U)):
                reconstructed = torch.tensordot(reconstructed, U[i], dims=([0], [1]))

            reg_factor = 1e-4
            loss = loss_fn(reconstructed, data) + reg_factor * (torch.norm(G) + sum(torch.norm(u) for u in U))
            loss.backward()
            return loss

        loss = optimizer.step(closure)
        
        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item():.2f}")

    reconstructed = G.clone()
    for i in range(len(U)):
        reconstructed = torch.tensordot(reconstructed, U[i], dims=([0], [1]))

    return reconstructed, G, U


In [93]:
rank = (10, 10, 10)
num_epochs = 1000
lr = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"

restored_tensor, G_optimized, U_optimized = optimize_tucker(data_noisy, rank, num_epochs, lr, device)

mse = torch.mean((restored_tensor - data_noisy) ** 2).item()
print(f"MSE после оптимизации: {mse:.2f}")


Epoch 0/1000, Loss: 1002.97
Epoch 100/1000, Loss: 400.84
Epoch 200/1000, Loss: 230.54
Epoch 300/1000, Loss: 154.10
Epoch 400/1000, Loss: 111.97
Epoch 500/1000, Loss: 85.82
Epoch 600/1000, Loss: 68.30
Epoch 700/1000, Loss: 55.88
Epoch 800/1000, Loss: 46.71
Epoch 900/1000, Loss: 39.72
MSE после оптимизации: 34.25


## 5 Приведите сравнение скорости работы и ошибки восстановления методом из пакета и реализованного градиентного
Сравнение может считаться ± объективным с размером выборки от 10.

In [94]:
def compare_methods(data, rank, num_epochs=1000, lr=1e-3, device="cuda"):
    start_time = time.time()
    tl_data = tl.tensor(data.numpy())
    core, factors = tucker(tl_data, rank)
    restored_tl = tl.tucker_to_tensor((core, factors))
    tl_time = time.time() - start_time
    tl_mse = mean_squared_error(data.numpy().flatten(), restored_tl.flatten())
    
    print(f"Tensorly: Время={tl_time:.4f} сек, MSE={tl_mse:.6f}")

    start_time = time.time()
    restored_grad, _, _ = optimize_tucker(data, rank, num_epochs, lr, device)
    grad_time = time.time() - start_time
    grad_mse = torch.mean((restored_grad - data) ** 2).item()

    print(f"Градиентный метод: Время={grad_time:.4f} сек, MSE={grad_mse:.6f}")

    results = {
        "Method": ["Tensorly", "Gradient"],
        "Time (sec)": [tl_time, grad_time],
        "MSE": [tl_mse, grad_mse],
    }

    return results

rank = (10, 10, 10)
num_epochs = 1000
lr = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"

comparison_results = compare_methods(data_noisy, rank, num_epochs, lr, device)


Tensorly: Время=2.4710 сек, MSE=0.000000
Epoch 0/1000, Loss: 982.88
Epoch 100/1000, Loss: 389.03
Epoch 200/1000, Loss: 223.34
Epoch 300/1000, Loss: 149.14
Epoch 400/1000, Loss: 108.27
Epoch 500/1000, Loss: 82.93
Epoch 600/1000, Loss: 65.93
Epoch 700/1000, Loss: 53.89
Epoch 800/1000, Loss: 45.01
Epoch 900/1000, Loss: 38.24
Градиентный метод: Время=30.5879 сек, MSE=32.929582
